# Segmenting and Clustering Neighbourhoods in Toronto

### Part 1: Scraping Postal Codes from Wikipidea

In [1]:
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request

import numpy as np
import pandas as pd

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

### The following two python functions are used to scrape html tables using
* __BeautifulSoup__
* __lxml__

In [3]:
def scrape_table_bs4(cname,cols):
    page  = urllib.request.urlopen(url).read()
    soup  = bs.BeautifulSoup(page,'lxml')
    table = soup.find("table",class_=cname)
    header = [head.findAll(text=True)[0].strip() for head in table.find_all("th")]
    data   = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]
              for tr in table.find_all("tr")]
    data    = [row for row in data if len(row) == cols]
    # Store data to this temporary dataframe
    raw_df = pd.DataFrame(data,columns=header)
    return raw_df

def scrape_table_lxml(XPATH,cols):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    table_content = doc.xpath(XPATH)
    for table in table_content:
        headers = [th.text_content().strip() for th in table.xpath('//th')]
        headers = headers[0:3]
        data    = [[td.text_content().strip() for td in tr.xpath('td')] 
                   for tr in table.xpath('//tbody/tr')]
        data    = [row for row in data if len(row) == cols]
        raw_df = pd.DataFrame(data,columns=headers)
        return raw_df
    
raw_TorontoPostalCodes = scrape_table_bs4("wikitable",3)
raw_TorontoPostalCodes.columns = ['PostalCode','Borough','Neighbourhood']

### Cleaning up Data

In [4]:
TorontoPostalCodes=raw_TorontoPostalCodes[~raw_TorontoPostalCodes['Borough'].isin(['Not assigned'])]

TorontoPostalCodes=TorontoPostalCodes.sort_values(by=['PostalCode','Borough','Neighbourhood'], ascending=[1,1,1]).reset_index(drop=True)

TorontoPostalCodes.loc[TorontoPostalCodes['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = TorontoPostalCodes['Borough']

TorontoPostalCodes = TorontoPostalCodes.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(",".join).reset_index()

In [5]:
print('Final Shape of Toronto Postal Codes Data Frame is:',TorontoPostalCodes.shape)
TorontoPostalCodes.head()

Final Shape of Toronto Postal Codes Data Frame is: (103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Part 2: Getting Latitude and Longitude for each Postal Code

In [6]:
toronto_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv
geo_data = pd.read_csv(toronto_geocsv)
geo_data.columns = ['PostalCode','Latitude','Longitude']
print(geo_data.head())

  PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476


In [7]:
TorontoPostalCodes = pd.merge(TorontoPostalCodes,geo_data,on = 'PostalCode')
print(TorontoPostalCodes.shape)
print(TorontoPostalCodes.head())

(103, 5)
  PostalCode      Borough                             Neighbourhood  \
0        M1B  Scarborough                           Malvern / Rouge   
1        M1C  Scarborough  Rouge Hill / Port Union / Highland Creek   
2        M1E  Scarborough       Guildwood / Morningside / West Hill   
3        M1G  Scarborough                                    Woburn   
4        M1H  Scarborough                                 Cedarbrae   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476  


### Part 3: Exploring and clustering the Neighbourhoods in Toronto

In [8]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [9]:
address = 'Toronto, Ontario Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto Canada are 43.6534817, -79.3839347.


In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(TorontoPostalCodes['Latitude'], TorontoPostalCodes['Longitude'], TorontoPostalCodes['Borough'], TorontoPostalCodes['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
toronto_data = TorontoPostalCodes[TorontoPostalCodes['Borough'].str.contains("Toronto")]
toronto_data = toronto_data.drop('PostalCode', axis = 1)
print(toronto_data.shape)
toronto_data.head()

(39, 4)


,Borough,Neighbourhood,Latitude,Longitude
37,East Toronto,The Beaches,43.676357,-79.293031
41,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
42,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
43,East Toronto,Studio District,43.659526,-79.340923
44,Central Toronto,Lawrence Park,43.728020,-79.388790


In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [13]:
CLIENT_ID = 'RM3JKWHSNFHQPYK13NRR440FSH2VR032HQYD3LROQ01CATN5' # my Foursquare ID
CLIENT_SECRET = 'A5UUUDDRTBY32X4EGOJ12BMOPIRQCI1KW5QFNYZVBSYHL3JC' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RM3JKWHSNFHQPYK13NRR440FSH2VR032HQYD3LROQ01CATN5
CLIENT_SECRET:A5UUUDDRTBY32X4EGOJ12BMOPIRQCI1KW5QFNYZVBSYHL3JC


#### Let's explore the first Neighbourhood in our dataframe.

Get the Neighbourhood's name.

In [16]:
toronto_data['Neighbourhood'].iloc[0]

'The Beaches'

Get the Neighbourhood's latitude and longitude values.

In [17]:
Neighbourhood_latitude = toronto_data['Latitude'].iloc[0] # Neighbourhood latitude value
Neighbourhood_longitude = toronto_data['Longitude'].iloc[0] # Neighbourhood longitude value

Neighbourhood_name = toronto_data['Neighbourhood'].iloc[0] # Neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(Neighbourhood_name, 
                                                               Neighbourhood_latitude, 
                                                               Neighbourhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


### Part 3(a): Explore Neighbourhoods in toronto

#### Let's create a function to repeat the same process to all the Neighbourhoods in toronto

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each Neighbourhood and create a new dataframe called *toronto_venues*.

In [19]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
Roselawn
Forest Hill North & West
The Annex / North Midtown / Yorkville
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst
Stn A PO Boxes
First Canadian Place / Underground city
Christie
Dufferin / Dovercourt Village
Little Portugal / Trinity
Brockton / Parkdale Village / Exhibition Place
High Park / The Junction South
Parkdale / Roncesva

#### Let's check the size of the resulting dataframe

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(866, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth West / Riverdale,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Let's check how many venues were returned for each Neighbourhood

In [21]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,30,30,30,30,30,30
Brockton / Parkdale Village / Exhibition Place,22,22,22,22,22,22
Business reply mail Processing CentrE,18,18,18,18,18,18
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst,16,16,16,16,16,16
Central Bay Street,30,30,30,30,30,30
Christie,18,18,18,18,18,18
Church and Wellesley,30,30,30,30,30,30
Commerce Court / Victoria Hotel,30,30,30,30,30,30
Davisville,30,30,30,30,30,30


#### Let's find out how many unique categories can be curated from all the returned venues

In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 193 uniques categories.


<a id='item3'></a>

### Part 3(b): Analysing Each Neighbourhood

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move Neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Danforth West / Riverdale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [24]:
toronto_onehot.shape

(866, 194)

#### Next, let's group rows by Neighbourhood and by taking the mean of the frequency of occurrence of each category

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
3,CN Tower / King and Spadina / Railway Lands / ...,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
5,Christie,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Commerce Court / Victoria Hotel,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.033333,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### Let's confirm the new size

In [26]:
toronto_grouped.shape

(39, 194)

#### Let's print each Neighbourhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1  Seafood Restaurant  0.07
2      Farmers Market  0.07
3              Bakery  0.07
4                Café  0.07


----Brockton / Parkdale Village / Exhibition Place----
            venue  freq
0            Café  0.14
1     Coffee Shop  0.09
2  Breakfast Spot  0.09
3             Gym  0.05
4          Bakery  0.05


----Business reply mail Processing CentrE----
                  venue  freq
0    Light Rail Station  0.11
1           Yoga Studio  0.06
2  Fast Food Restaurant  0.06
3                  Park  0.06
4           Pizza Place  0.06


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst----
                venue  freq
0      Airport Lounge  0.12
1     Airport Service  0.12
2             Airport  0.06
3  Airport Food Court  0.06
4                 Bar  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.23
1                  Spa  0.07
2  Japanese Re

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each Neighbourhood.

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
Neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    Neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

Neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Café,Beer Bar,Cocktail Bar,Farmers Market,Bakery,Park,Breakfast Spot,Bistro
1,Brockton / Parkdale Village / Exhibition Place,Café,Breakfast Spot,Coffee Shop,Pet Store,Italian Restaurant,Restaurant,Burrito Place,Stadium,Bar,Bakery
2,Business reply mail Processing CentrE,Light Rail Station,Yoga Studio,Spa,Fast Food Restaurant,Farmers Market,Comic Shop,Garden Center,Park,Pizza Place,Burrito Place
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Lounge,Airport Service,Airport,Rental Car Location,Plane,Harbor / Marina,Boat or Ferry,Coffee Shop,Boutique,Sculpture Garden
4,Central Bay Street,Coffee Shop,Spa,Italian Restaurant,Japanese Restaurant,Seafood Restaurant,Bubble Tea Shop,Miscellaneous Shop,Sandwich Place,Ice Cream Shop,Poke Place


<a id='item4'></a>

### Part 3(c): Clustering Neighbourhoods

Run *k*-means to cluster the Neighbourhood into 5 clusters.

In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each Neighbourhood.

In [31]:
# add clustering labels
Neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each Neighbourhood
toronto_merged = toronto_merged.join(Neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Neighborhood,Trail,Pub,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
41,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188,1,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Brewery,Bubble Tea Shop,Bookstore,Diner,Juice Bar,Dessert Shop
42,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572,1,Pizza Place,Sandwich Place,Park,Fast Food Restaurant,Italian Restaurant,Movie Theater,Board Shop,Ice Cream Shop,Burrito Place,Pub
43,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Bar,Ice Cream Shop,Sandwich Place,Bookstore,Brewery,Italian Restaurant,Yoga Studio
44,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Creperie,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


Finally, let's visualize the resulting clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

### Part 3(d): Examining Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,The Beaches,Health Food Store,Neighborhood,Trail,Pub,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


#### Cluster 2

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,The Danforth West / Riverdale,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Brewery,Bubble Tea Shop,Bookstore,Diner,Juice Bar,Dessert Shop
42,India Bazaar / The Beaches West,Pizza Place,Sandwich Place,Park,Fast Food Restaurant,Italian Restaurant,Movie Theater,Board Shop,Ice Cream Shop,Burrito Place,Pub
43,Studio District,Café,Coffee Shop,Bakery,Bar,Ice Cream Shop,Sandwich Place,Bookstore,Brewery,Italian Restaurant,Yoga Studio
45,Davisville North,Park,Gym,Breakfast Spot,Sandwich Place,Department Store,Hotel,Food & Drink Shop,Comic Shop,Colombian Restaurant,Discount Store
46,North Toronto West,Coffee Shop,Clothing Store,Sporting Goods Shop,Miscellaneous Shop,Mexican Restaurant,Park,Chinese Restaurant,Pet Store,Café,Rental Car Location
47,Davisville,Dessert Shop,Coffee Shop,Sushi Restaurant,Pizza Place,Café,Italian Restaurant,Sandwich Place,Gym,Park,Pharmacy
49,Summerhill West / Rathnelly / South Hill / For...,Coffee Shop,Pub,Pizza Place,Restaurant,Bank,Sports Bar,Bagel Shop,Light Rail Station,Supermarket,Liquor Store
51,St. James Town / Cabbagetown,Restaurant,Italian Restaurant,Coffee Shop,Café,Playground,Bakery,Bank,Caribbean Restaurant,Jewelry Store,Beer Store
52,Church and Wellesley,Coffee Shop,Gay Bar,Burger Joint,Bookstore,Pub,Diner,Ice Cream Shop,Salon / Barbershop,Indian Restaurant,Restaurant
53,Regent Park / Harbourfront,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Mexican Restaurant,Yoga Studio,Distribution Center,Restaurant,Spa


#### Cluster 3

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Moore Park / Summerhill East,Restaurant,Yoga Studio,Dumpling Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


#### Cluster 4

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Roselawn,Garden,Pool,Yoga Studio,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


#### Cluster 5

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Lawrence Park,Park,Swim School,Bus Line,Creperie,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
50,Rosedale,Park,Trail,Playground,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
64,Forest Hill North & West,Park,Bus Line,Jewelry Store,Trail,Sushi Restaurant,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
